In [ ]:
import glob
import numpy as np
import pandas as pd 
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LogNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib import rcParams

print('xarray = ', xr.__version__)
print('matplotlib = ', matplotlib.__version__)

almost_zero=1.0e-6
clon=180

land_10m = cfeature.NaturalEarthFeature(
    "physical", "land", "10m", facecolor=cfeature.COLORS["land"]
)

def plot_decor(ax):
    ax.coastlines("10m")
    ax.set_extent([172,180,-42,-34])
    ax.add_feature(land_10m, zorder=-1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
def compute_arrival_time(dain, vmin=0.001):
    if dain.dims[0] != 'time':
        print(dain.shape)
        print('bad shape. Shoud be time, lat, lon')
        raise
    nt = dain.sizes['time']
    daout = xr.where(dain > vmin, 1., 0)
    daout = daout.cumsum(dim='time')
    daout = daout.where(daout > 0)
    # Assuming hourly data
    for it in range(nt):
         daout.data[it,...] = it+2 - daout.isel(time=it).data
    return daout


# Set customizable global variables

In [ ]:
density = 1300 # kg/m3 andesite for Tongariro
it = -1  # plots for last time instant in files (same as it=23)

savefigs = True
fig_args = dict(bbox_inches='tight', dpi=300)

# UNCOMMENT FOR OPERATIONAL CASE STUDY
file_lhs = '202107150000/input/lh_sample_intermediate_n30_1.csv'
data_pattern = '202107150000/output/*Tongariro*nc'
fileplot_prefix = 'plt_det_202107150000_Tongariro'

# UNCOMMENT FOR HISTORICAL CASE STUDY
# file_lhs = '201208061200/input/lh_sample_intermediate_n20_1.csv'
# data_pattern = '201208061200/output/*Tongariro*nc'
# fileplot_prefix = 'plt_det_201208061200_Tongariro'

# Read files

In [ ]:
files = glob.glob(data_pattern)
files.sort()
print(len(files))

for x in files[:3]:
    print(x)

In [ ]:
ds = xr.open_mfdataset(files, combine='nested', concat_dim=['idx'])
ds

In [ ]:
vlon, vlat = ds.attrs['src_lon'], ds.attrs['src_lat']
vlon, vlat

## Add arrival time

In [ ]:
ds.chunks

In [ ]:
nidx = ds.sizes['idx']
ds['arrival_time'] = ds['total_deposition'].copy(deep=True)
ds['arrival_time'].attrs['long_name'] = 'arrival time'
ds['arrival_time'].attrs['units'] = 'hours'
for idx in range(nidx):
    da = compute_arrival_time(ds['total_deposition'].isel(idx=idx))
    ds['arrival_time'].data[idx,...] = da

In [ ]:
ds.chunks

# Read scenarios

In [ ]:
scenarios = pd.read_csv(file_lhs)
scenarios = scenarios.rename({'MER [kg/s]': 'MER', 'Column height [km]': 'H', 'Duration [h]': 'D'}, axis=1)

scenarios['MER'] = scenarios['MER']/1.e6 # MegaTons

# total mass erupted
scenarios['ME'] = scenarios['MER'] * scenarios['D']

scenarios.head()

In [ ]:
# Print max_ash_depth for each scenario just to have an idea of the range
ds_slice = ds['total_deposition'].isel(time=-1)
max_ash_depth = []
for i in range(ds_slice.sizes['idx']):
    da = ds_slice.isel(idx=i)    
    #print(i, np.round(np.nanmin(da),0),np.nanmax(da))
    max_ash_depth += [np.nanmax(da)]
    

# Adding max ash depth computed above
scenarios['max_ash_depth'] = max_ash_depth
scenarios

In [ ]:
axs = pd.plotting.scatter_matrix(scenarios, diagonal='kde',figsize=(12,10), marker='o')

# Plots

## Define levels and colormaps

In [ ]:
lev_max = 1e3
lev_min = 1e-3
lev_exp = np.arange(np.log10( lev_min), np.log10(lev_max).max() + 1)
print(lev_exp)
levs = np.power(10, lev_exp)
print(levs)


def mycolormap(cmap_name, n):
    cmap = plt.cm.get_cmap(cmap_name, len(levs)+1)
    colors = list(cmap(np.arange(len(levs)+1)))
    # replace first color with transparent
    colors[0][-1] = 0
    cmap2 = ListedColormap(colors)
    # set over-color to last color of list 
#     cmap2.set_over(colors[-1])
    return cmap2

cmap = plt.cm.get_cmap('hot_r', len(levs)+1)
cmap2 = mycolormap('hot_r', len(levs)+1)

In [ ]:
cmap

In [ ]:
cmap2

## Plot ash depth + arrival time for each scenario

In [ ]:
for idx in range(nidx):
    scenario = scenarios.iloc[idx]
    da = ds['total_deposition'].isel(idx=idx, time=it)
    da_arrival = ds['arrival_time'].isel(idx=idx, time=it)
    levels_arrival = np.arange(24)
    
    fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})
    p1 = da.plot.contourf(ax=ax, levels=levs, cmap=cmap2, norm=LogNorm(vmin=levs[0], vmax=levs[-1]),
        extend='min',
        transform=ccrs.PlateCarree(),
        cbar_kwargs={'label': 'Accumulated ashfall thickness [mm]'},
    )

    # contours after zooming..
    try:
        cs = da_arrival.plot.contour(ax=ax, levels=levels_arrival, colors='white',
            transform=ccrs.PlateCarree(),
        )
        plt.clabel(cs, levels=levels_arrival, fmt='%0d h') 
    except ValueError as exc:
        print(exc)
        print('Skipping...')
        
    ax.plot(vlon,vlat, transform=ccrs.PlateCarree(), marker='*', ms=8, color='k')
    plot_decor(ax)
    txtstr = 'Scenario {}: MER={MER:.2f} MT/h; H={H:.2f} km; D={D:.2f} h'.format(idx, **scenario)
    ax.set_title(txtstr)
    
    fileplot = fileplot_prefix + '_ashfall_arrival_idx{:03d}.png'.format(idx)
    print(fileplot)
    fig.savefig(fileplot, **fig_args)
    #plt.close()
    

## Plot ash depth for each scenario (for paper)

In [ ]:
for idx in range(nidx):
    scenario = scenarios.iloc[idx]
    da = ds['total_deposition'].isel(idx=idx, time=it)
    
    fig, ax = plt.subplots(1,1, figsize=(12,10), subplot_kw={"projection": ccrs.PlateCarree(clon)})
    p1 = da.plot.contourf(ax=ax, levels=levs, cmap=cmap2, norm=LogNorm(vmin=levs[0], vmax=levs[-1]), 
        extend='min',
        transform=ccrs.PlateCarree(),
        cbar_kwargs={'label': 'Accumulated ashfall thickness [mm]'},
    )

    ax.plot(vlon,vlat, transform=ccrs.PlateCarree(), marker='*', ms=8, color='k')
    plot_decor(ax)
    txtstr = 'Scenario {}: MER={MER:.2f} MT/h; H={H:.2f} km; D={D:.2f} h'.format(idx, **scenario)
    ax.set_title(txtstr)
    
    fileplot = fileplot_prefix + '_ashfall_idx{:03d}.png'.format(idx)
    print(fileplot)
    fig.savefig(fileplot, **fig_args)
    #plt.close()
    

In [ ]:
rcParams['figure.subplot.hspace'] = 0.01
fontsize = 17
fig, axs = plt.subplot_mosaic([['a) ', 'b) '], ['c) ', 'd) ']], figsize=(21,21), subplot_kw={"projection": ccrs.PlateCarree(clon)})
idx2ax = {0: 'a) ', 4: 'b) ', 6: 'c) ', 15: 'd) '}
images = []

for idx in [0, 4, 6, 15]:
    scenario = scenarios.iloc[idx]
    da = ds['total_deposition'].isel(idx=idx, time=it)
    axl = idx2ax[idx]
    p1 = da.plot.contourf(ax=axs[axl], levels=levs, cmap=cmap2, norm=LogNorm(vmin=levs[0], vmax=levs[-1]), 
        extend='min',
        transform=ccrs.PlateCarree(),
        add_colorbar=False
    )
    images.append(p1)
    axs[axl].plot(vlon,vlat, transform=ccrs.PlateCarree(), marker='*', ms=8, color='k')
    plot_decor(axs[axl])
    txtstr = '  Scenario {}: MER={MER:.1f} MT/h; H={H:.1f} km; D={D:.1f} h'.format(idx, **scenario)
    axs[axl].set_title(txtstr, fontsize=fontsize)
    axs[axl].set_title(axl, fontfamily='serif', loc='left', fontsize=fontsize)
    
vmin = min(image.get_array().min() for image in images)
vmax = max(image.get_array().max() for image in images)
norm = LogNorm(vmin=vmin, vmax=vmax)
for im in images:
    im.set_norm(norm)
    
cax = fig.colorbar(images[0], ax=fig.axes, format="%g", orientation='vertical', fraction=.05, shrink=0.6)
cax.ax.tick_params(axis='both', labelsize=fontsize)
cax.set_label(label='Accumulated ashfall thickness [mm]', fontsize=fontsize)

fileplot = 'Figure07.png'
fig.savefig(fileplot, **fig_args)
